# DYD IECC Locations

## Preprocess

1. Downloaded Geojson from IECC

2. Got city/states based on zip code

3. Merged on fips.

4. Export for work in Tableau

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime
import requests
import json
import geojson
import geopandas as gpd

---
## Read in CSV

In [2]:
# Import the CSV that has the city field to be added to the geojson.
stateCity = pd.read_csv("../Support_Files/uscities.csv")

stateCity

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,Prattville,Prattville,AL,Alabama,1001,Autauga,32.4605,-86.4588,35957,410,polygon,False,True,America/Chicago,3,36066 36067 36068,1840004086
1,Pine Level,Pine Level,AL,Alabama,1001,Autauga,32.5797,-86.4529,4484,69,polygon,False,False,America/Chicago,3,36067 36022,1840023046
2,Marbury,Marbury,AL,Alabama,1001,Autauga,32.6793,-86.4607,1633,27,polygon,False,False,America/Chicago,3,36022 36051,1840023045
3,Autaugaville,Autaugaville,AL,Alabama,1001,Autauga,32.4320,-86.6579,871,42,polygon,False,True,America/Chicago,3,36003,1840015797
4,Billingsley,Billingsley,AL,Alabama,1001,Autauga,32.6610,-86.7087,142,46,polygon,False,True,America/Chicago,3,36006,1840015798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28333,El Negro,El Negro,PR,Puerto Rico,72151,Yabucoa,18.0370,-65.8494,1613,1957,polygon,False,False,America/Puerto_Rico,3,767,1630023684
28334,Comunas,Comunas,PR,Puerto Rico,72151,Yabucoa,18.0871,-65.8404,758,468,polygon,False,False,America/Puerto_Rico,3,767,1630035651
28335,Rosa Sánchez,Rosa Sanchez,PR,Puerto Rico,72151,Yabucoa,18.0623,-65.9141,607,538,polygon,False,False,America/Puerto_Rico,3,767,1630023681
28336,Yauco,Yauco,PR,Puerto Rico,72153,Yauco,18.0344,-66.8615,16585,2211,polygon,False,False,America/Puerto_Rico,2,698,1630035600


In [3]:
# rename "county_fips" to "FIPS"

stateCity = stateCity.rename(columns={"county_fips" : "FIPS"})
stateCity = stateCity.rename(columns={"county_name" : "NAME"})

stateCity

,city,city_ascii,state_id,state_name,FIPS,NAME,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,Prattville,Prattville,AL,Alabama,1001,Autauga,32.4605,-86.4588,35957,410,polygon,False,True,America/Chicago,3,36066 36067 36068,1840004086
1,Pine Level,Pine Level,AL,Alabama,1001,Autauga,32.5797,-86.4529,4484,69,polygon,False,False,America/Chicago,3,36067 36022,1840023046
2,Marbury,Marbury,AL,Alabama,1001,Autauga,32.6793,-86.4607,1633,27,polygon,False,False,America/Chicago,3,36022 36051,1840023045
3,Autaugaville,Autaugaville,AL,Alabama,1001,Autauga,32.4320,-86.6579,871,42,polygon,False,True,America/Chicago,3,36003,1840015797
4,Billingsley,Billingsley,AL,Alabama,1001,Autauga,32.6610,-86.7087,142,46,polygon,False,True,America/Chicago,3,36006,1840015798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28333,El Negro,El Negro,PR,Puerto Rico,72151,Yabucoa,18.0370,-65.8494,1613,1957,polygon,False,False,America/Puerto_Rico,3,767,1630023684
28334,Comunas,Comunas,PR,Puerto Rico,72151,Yabucoa,18.0871,-65.8404,758,468,polygon,False,False,America/Puerto_Rico,3,767,1630035651
28335,Rosa Sánchez,Rosa Sanchez,PR,Puerto Rico,72151,Yabucoa,18.0623,-65.9141,607,538,polygon,False,False,America/Puerto_Rico,3,767,1630023681
28336,Yauco,Yauco,PR,Puerto Rico,72153,Yauco,18.0344,-66.8615,16585,2211,polygon,False,False,America/Puerto_Rico,2,698,1630035600


In [4]:
# remove unwanted columns

stateCityReduced = stateCity.drop(["state_id", "state_name", "city_ascii", "population", "density", "military", "incorporated", "source", "ranking", "timezone", "id", "lat", "lng", "zips"], axis = 1)

stateCityReduced

,city,FIPS,NAME
0,Prattville,1001,Autauga
1,Pine Level,1001,Autauga
2,Marbury,1001,Autauga
3,Autaugaville,1001,Autauga
4,Billingsley,1001,Autauga
...,...,...,...
28333,El Negro,72151,Yabucoa
28334,Comunas,72151,Yabucoa
28335,Rosa Sánchez,72151,Yabucoa
28336,Yauco,72153,Yauco


In [5]:
stateCityReduced.dtypes


city    object
FIPS     int64
NAME    object
dtype: object

In [6]:
#convert int64 to string
stateCityReduced["FIPS"] = str(stateCityReduced["FIPS"])
stateCityReduced["city"] = str(stateCityReduced["city"])
stateCityReduced["NAME"] = str(stateCityReduced["NAME"])

In [7]:
stateCityReduced.dtypes

city    object
FIPS    object
NAME    object
dtype: object

## Read in the geojson

In [8]:
# Import the geojson.

IECCmap = gpd.read_file("../Support_Files/Building_America_and_IECC_Climate_Zones_by_U_S__County_Boundaries_(Detailed).geojson")

IECCmap

,OBJECTID,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,POP2010,POP10_SQMI,POP2013,POP13_SQMI,...,SQMI,Shape__Area,Shape__Length,State,IECC_Climate_Zone,IECC_Moisture_Regime,BA_Climate_Zone,County_Name,Temp,geometry
0,1,Falls Church,Virginia,51,610,51610,12332,6197.0,13145,6605.5,...,1.99,0.000535,0.117034,VA,4,A,Mixed-Humid,Falls Church (city),1.988348,"POLYGON ((-77.17572 38.89376, -77.17567 38.893..."
1,2,Lexington,Virginia,51,678,51678,7042,2805.6,7048,2808.0,...,2.51,0.000664,0.142632,VA,4,A,Mixed-Humid,Lexington (city),2.507380,"POLYGON ((-79.43371 37.79363, -79.43192 37.793..."
2,3,Manassas Park,Virginia,51,685,51685,14273,5575.4,14856,5803.1,...,2.56,0.000687,0.172682,VA,4,A,Mixed-Humid,Manassas Park (city),2.558983,"POLYGON ((-77.45911 38.78154, -77.45942 38.778..."
3,4,Bedford City,Virginia,51,515,51515,6222,901.7,6353,920.7,...,6.90,0.001817,0.216277,VA,4,A,Mixed-Humid,Bedford (city),6.899402,"POLYGON ((-79.50116 37.35577, -79.49983 37.354..."
4,5,Covington,Virginia,51,580,51580,5961,1055.0,6124,1083.9,...,5.65,0.001497,0.228160,VA,4,A,Mixed-Humid,Covington (city),5.650342,"POLYGON ((-79.98433 37.81063, -79.98399 37.806..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3214,3215,Bethel,Alaska,02,050,02050,17013,0.4,17189,0.4,...,41706.31,17.872409,65.993987,AK,8,N/A,Subarctic,Bethel,41703.764979,"MULTIPOLYGON (((-162.06856 60.60158, -162.0680..."
3215,3216,Kodiak Island,Alaska,02,150,02150,13592,2.1,14135,2.1,...,6627.06,2.582962,66.825231,AK,7,N/A,Very Cold,Kodiak Island,6626.705734,"MULTIPOLYGON (((-154.12944 56.74231, -154.1375..."
3216,3217,Nome,Alaska,02,180,02180,9492,0.4,9711,0.4,...,23079.20,11.322305,77.113078,AK,8,N/A,Subarctic,Nome,23077.596757,"MULTIPOLYGON (((-161.99091 63.01268, -161.9909..."
3217,3218,Valdez-Cordova,Alaska,02,261,02261,9636,0.3,9940,0.3,...,34899.89,15.266692,85.755667,AK,7,N/A,Very Cold,Valdez-Cordova,34897.711626,"MULTIPOLYGON (((-144.66792 60.28286, -144.6698..."


In [9]:
IECCmap["FIPS"].dtypes

dtype('O')

In [10]:
# #convert int64 to string
# IECCmap["FIPS"] = int(IECCmap["FIPS"])

# IECCmap["FIPS"].dtypes

In [11]:
# merge in the city name field from the CSV file

IECCmapMerge = gpd.GeoDataFrame(pd.merge(IECCmap, stateCityReduced, on=["NAME", "FIPS"], how="left"))

In [12]:
IECCmapMerge

,OBJECTID,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,POP2010,POP10_SQMI,POP2013,POP13_SQMI,...,Shape__Area,Shape__Length,State,IECC_Climate_Zone,IECC_Moisture_Regime,BA_Climate_Zone,County_Name,Temp,geometry,city
0,1,Falls Church,Virginia,51,610,51610,12332,6197.0,13145,6605.5,...,0.000535,0.117034,VA,4,A,Mixed-Humid,Falls Church (city),1.988348,"POLYGON ((-77.17572 38.89376, -77.17567 38.893...",NaN
1,2,Lexington,Virginia,51,678,51678,7042,2805.6,7048,2808.0,...,0.000664,0.142632,VA,4,A,Mixed-Humid,Lexington (city),2.507380,"POLYGON ((-79.43371 37.79363, -79.43192 37.793...",NaN
2,3,Manassas Park,Virginia,51,685,51685,14273,5575.4,14856,5803.1,...,0.000687,0.172682,VA,4,A,Mixed-Humid,Manassas Park (city),2.558983,"POLYGON ((-77.45911 38.78154, -77.45942 38.778...",NaN
3,4,Bedford City,Virginia,51,515,51515,6222,901.7,6353,920.7,...,0.001817,0.216277,VA,4,A,Mixed-Humid,Bedford (city),6.899402,"POLYGON ((-79.50116 37.35577, -79.49983 37.354...",NaN
4,5,Covington,Virginia,51,580,51580,5961,1055.0,6124,1083.9,...,0.001497,0.228160,VA,4,A,Mixed-Humid,Covington (city),5.650342,"POLYGON ((-79.98433 37.81063, -79.98399 37.806...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3214,3215,Bethel,Alaska,02,050,02050,17013,0.4,17189,0.4,...,17.872409,65.993987,AK,8,N/A,Subarctic,Bethel,41703.764979,"MULTIPOLYGON (((-162.06856 60.60158, -162.0680...",NaN
3215,3216,Kodiak Island,Alaska,02,150,02150,13592,2.1,14135,2.1,...,2.582962,66.825231,AK,7,N/A,Very Cold,Kodiak Island,6626.705734,"MULTIPOLYGON (((-154.12944 56.74231, -154.1375...",NaN
3216,3217,Nome,Alaska,02,180,02180,9492,0.4,9711,0.4,...,11.322305,77.113078,AK,8,N/A,Subarctic,Nome,23077.596757,"MULTIPOLYGON (((-161.99091 63.01268, -161.9909...",NaN
3217,3218,Valdez-Cordova,Alaska,02,261,02261,9636,0.3,9940,0.3,...,15.266692,85.755667,AK,7,N/A,Very Cold,Valdez-Cordova,34897.711626,"MULTIPOLYGON (((-144.66792 60.28286, -144.6698...",NaN


In [13]:
print(IECCmapMerge.columns)

Index(['OBJECTID', 'NAME', 'STATE_NAME', 'STATE_FIPS', 'CNTY_FIPS', 'FIPS',
       'POP2010', 'POP10_SQMI', 'POP2013', 'POP13_SQMI', 'WHITE', 'BLACK',
       'AMERI_ES', 'ASIAN', 'HAWN_PI', 'HISPANIC', 'OTHER', 'MULT_RACE',
       'MALES', 'FEMALES', 'AGE_UNDER5', 'AGE_5_9', 'AGE_10_14', 'AGE_15_19',
       'AGE_20_24', 'AGE_25_34', 'AGE_35_44', 'AGE_45_54', 'AGE_55_64',
       'AGE_65_74', 'AGE_75_84', 'AGE_85_UP', 'MED_AGE', 'MED_AGE_M',
       'MED_AGE_F', 'HOUSEHOLDS', 'AVE_HH_SZ', 'HSEHLD_1_M', 'HSEHLD_1_F',
       'MARHH_CHD', 'MARHH_NO_C', 'MHH_CHILD', 'FHH_CHILD', 'FAMILIES',
       'AVE_FAM_SZ', 'HSE_UNITS', 'VACANT', 'OWNER_OCC', 'RENTER_OCC',
       'NO_FARMS12', 'AVE_SIZE12', 'CROP_ACR12', 'AVE_SALE12', 'SQMI',
       'Shape__Area', 'Shape__Length', 'State', 'IECC_Climate_Zone',
       'IECC_Moisture_Regime', 'BA_Climate_Zone', 'County_Name', 'Temp',
       'geometry', 'city'],
      dtype='object')


In [15]:
IECCmapMerge["city"].dtypes

dtype('O')